In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.8.17 torch-1.12.1+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
Setup complete ✅ (8 CPUs, 7.9 GB RAM, 199.5/221.5 GB disk)


In [2]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import numpy as np

In [4]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [6]:
import torch

if torch.cuda.is_available() == True:
    device = 'cuda:0'
else:
    device = 'cpu'

print(device)

cpu


In [5]:
class hand_LSTM(nn.Module):
    def __init__(self):
        super(hand_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size = 65, hidden_size = 128, num_layers=1, batch_first=True) # xyz 63 + x_center, y_center = 65
        self.lstm2 = nn.LSTM(input_size = 128, hidden_size = 256, num_layers=1, batch_first=True)
        self.lstm3 = nn.LSTM(input_size = 256, hidden_size = 512, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        self.lstm4 = nn.LSTM(input_size = 512, hidden_size = 256, num_layers=1, batch_first=True)
        self.lstm5 = nn.LSTM(input_size = 256, hidden_size = 128, num_layers=1, batch_first=True)
        self.lstm6 = nn.LSTM(input_size = 128, hidden_size = 64, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.lstm7 = nn.LSTM(input_size = 64, hidden_size = 32, num_layers=1, batch_first=True)
        self.fc = nn.Linear(32, 2)
    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:, -1,:])
        return x

In [8]:
# 모델 불러오기
model_path = './lstm_model.pth'

lstm_model = hand_LSTM()
lstm_model.load_state_dict(torch.load(model_path, map_location=device))
lstm_model.eval()
print("모델이 성공적으로 불러와졌습니다.")

모델이 성공적으로 불러와졌습니다.


In [9]:
# YOLO 객체 감지 모델 초기화
# best_model = '/content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/best.pt'
best_model = 'best.pt'
yolo_model = YOLO(best_model)

In [ ]:
# mediapipe 손 감지 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(1) # for Mac
# cap = cv2.VideoCapture(0) # for Windows

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR 이미지를 RGB로 변환
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(frame, 1)
    results = hands.process(image)

    # YOLO 객체 감지
    box_results = yolo_model.predict(image)
    boxes = box_results[0].boxes.xyxy.cpu()
    box_class = box_results[0].boxes.cls.cpu().tolist()
    confs = box_results[0].boxes.conf.float().cpu().tolist()


    # 객체 감지 결과를 이미지에 표시
    for box in boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
              with torch.no_grad():
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)


    cv2.imshow('Frame', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()